In [36]:
# https://towardsdatascience.com/hands-on-introduction-to-delta-lake-with-py-spark-b39460a4b1ae
import oci
from delta import *
import pyspark
from delta.tables import *
get_ipython().run_line_magic('load_ext', 'dataflow.magics')
variableofdatacience=""

The dataflow.magics extension is already loaded. To reload it, use:
  %reload_ext dataflow.magics


In [37]:
#sessionId='ocid1.dataflowapplication.oc1.ap-mumbai-1.anrg6ljrp77apcqargbozp4blsxq6j2ughe6tyfxft5xfj3zdizyduzcst7q' # GETGO Version
sessionId=''
%use_session -s {sessionId}

Using Active Session .. ocid1.dataflowrun.oc1.ap-mumbai-1.anrg6ljrp77apcqa67hohjygp2vktovwtjm5k7swctcbie557levi45vju2a


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cluster is ready..
Starting Spark application..


Session ID,Kind,State,Current session
ocid1.dataflowapplication.oc1.ap-mumbai-1.anrg6ljrp77apcqal54sk6dsgyp4ept2hol3sjuui3v73arise3zoi45vkna,pyspark,IN_PROGRESS,Dataflow Run


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
SparkContext available as 'sc'.


In [38]:
%%spark
print(spark)


storage_account_name=""


client_id=""
client_secret=""
directory_id=""

#Set config params described above
spark.conf.set("fs.azure.account.auth.type."+storage_account_name+".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type."+storage_account_name+".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id."+storage_account_name+".dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret."+storage_account_name+".dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint."+storage_account_name+".dfs.core.windows.net", "https://login.microsoftonline.com/"+directory_id+"/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

# spark.conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
# spark.conf.set("spark.delta.logStore.oci.impl" , "io.delta.storage.OracleCloudLogStore")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
%%spark 

path = ""
df = spark.read.format("csv").option("header","true").load(path)
df.show()
df.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|    starttime|     stoptime|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bikeid|  usertype|birth_year|gender|
+------------+-------------+-------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         975|09/01/16 0:00|09/01/16 0:16|             312|Allen St & Stanto...|             40.722055|             -73.989111|           313|Washington Ave & ...|         40.69610226|         -73.96751037| 22609|Subscriber|      1985|     1|
|        1220|09/01/16 0:00|

In [51]:
%%spark
# df.write.format("csv").option("header","true").save("oci://azure_dataflow@apaccpt03/demotohindalcocitibike.csv")
# df.write.format("delta").save("oci://azure_dataflow@apaccpt03/demotohindalcocitibikedelta")

# df.write.format("csv").option("header","true").save("oci://adw_spark@apaccpt03/demotohindalcocitibike.csv")
df.write.format("delta").save("oci://adw_spark@apaccpt03/demotogetgo")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
%%spark
deltadf=spark.read.format("delta").load("oci://azure_dataflow@apaccpt03/demotohindalcocitibikedelta").show()
df


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|     starttime|      stoptime|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bikeid|  usertype|birth_year|gender|
+------------+--------------+--------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         252|09/11/16 19:45|09/11/16 19:50|             168|     W 18 St & 6 Ave|           40.73971301|           -73.99456405|           382|University Pl & E...|         40.73492695|         -73.99200509| 17079|Subscriber|      1992|     1|
|         295|09/11/

In [35]:
%%spark
df.select(["tripduration"]).show(5)
#print(deltadf.columns())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|tripduration|
+------------+
|         975|
|        1220|
|         308|
|         250|
|         439|
+------------+
only showing top 5 rows

In [42]:
%%spark
newdf=(df.drop('tripduration'))
newdf.write.format("delta").mode("overwrite").save("oci://azure_dataflow@apaccpt03/newcoe")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
%%spark
csvdf.select(["tripduration"]).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|tripduration|
+------------+
|         975|
|        1220|
|         308|
|         250|
|         439|
+------------+
only showing top 5 rows

In [12]:
%%spark
newdf=(deltadf.drop('tripduration'))
newdf.write.format("delta").mode("overwrite").save("oci://azure_dataflow@apaccpt03/citibike_delta1019")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%spark
ndeltadf=spark.read.format("delta").option("versionAsOf", 1).load("oci://azure_dataflow@apaccpt03/citibike_delta1019")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%spark
ndeltadf.select(["tripduration"]).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|tripduration|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+
only showing top 5 rows

In [130]:
%%spark
newdf=(deltadf.drop(col("tripduration")))
newdf.write.format("delta").mode("overwrite").save("oci://azure_dataflow@apaccpt03/citibike_delta")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
%%spark
ndeltadf=spark.read.format("delta").option("versionAsOf", 5).load("oci://azure_dataflow@apaccpt03/citibike_delta")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
%%spark
ndeltadf.select(["tripduration"]).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|tripduration|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+
only showing top 5 rows